### https://dl.acm.org/doi/pdf/10.1145/2370776.2370782?casa_token=wd-hmH7FO4YAAAAA:EghGIClehmSaw35US0wrC8zc_JP0ODI69Zkd4snBHpQ4gLb-IB7DTVSKQI1P28qyrK-co0zfxy42

In [1]:
from clingo.symbol import Number, String
from clingo.control import Control

In [2]:
def run(axioms):
    evASP = Control("0")
    evASP.add("base", [], axioms)

    evASP.ground([("base", [])])
    
    with evASP.solve(yield_=True) as handle:
        #print(handle)
        for m in handle: 
            print("Answer:\n{}\n".format(m))
            #print(m)

## Test Data

In [81]:
# atoms
s = [0,"s"]
ns = [1,"s"]

t = [0,"t"]
nt = [1,"t"]

# rules
rul = [[0,"p"],[s,nt]]

# programs
prog = [rul]

print(prog)

[[[0, 'p'], [[0, 's'], [1, 't']]]]


## Checkt Even/Odd Loop Over Negation

In [82]:
def elon_or_olon(rule,program):
    
    elon = False
    olon = False
    
    query = rule[0]
    
    # root note
    paths = [[query[0]+atom[0], atom] for atom in rule[1]]
    
    # tree search
    while paths:
        
        # aggregates for next expansion
        tentative_expansion = []
        
        # cycle check and return updates
        for path in paths:
            # matches query
            if path[1][1] == query[1]:
                if (path[0] % 2) == 0:
                    elon = True
                else:
                    olon = True
                if elon and olon:
                    return [elon, olon]
            else:
                tentative_expansion.append(path) 
        
        if not tentative_expansion:
            break
            
        # breadth first search
        paths = [[path[0]+atom[0], atom] 
                 for path in tentative_expansion
                 for rule in program if rule[0] == path[1]
                 for atom in rule[1]]
        
    # non-cyclic case
    if olon: 
        return [False, olon]
    else: 
        return [True, olon]

In [83]:
elon_or_olon(rul,prog)

[True, False]

In [154]:
# p :- not q.  Rule P1.a
# q :- not p.  Rule P1.b

p = [0,"p"]
q = [0,"q"]
np = [1,"p"]
nq = [1,"q"]

rul1 = [p,[nq]]
rul2 = [q,[np]]

prog1 = [rul1,rul2]

print(prog1)

[[[0, 'p'], [[1, 'q']]], [[0, 'q'], [[1, 'p']]]]


In [155]:
elon_or_olon(rul1,prog1)

[True, False]

In [63]:
elon_or_olon(rul2,prog1)

[True, False]

In [68]:
# p :- q, not r.  Rule P3.a
# r :- not p.     Rule P3.b
# q :- t, not p.  Rule P3.c


p = [0,"p"]
q = [0,"q"]
r = [0,"r"]
t = [0,"t"]
np = [1,"p"]
nq = [1,"q"]
nr = [1,"r"]

rul3A = [p,[q,nr]]
rul3B = [r,[np]]
rul3C = [q,[t,np]]

prog3 = [rul3A,rul3B,rul3C]

print(prog3)

[[[0, 'p'], [[0, 'q'], [1, 'r']]], [[0, 'r'], [[1, 'p']]], [[0, 'q'], [[0, 't'], [1, 'p']]]]


In [69]:
elon_or_olon(rul3A,prog3)
### MIST

[False, True]

## For only ordinary rules

In [98]:
def coind(query,program,chs):
    
    
    matched_bodies = [rule[1]
                      for rule in program if query == rule[0]]
    
    
    answers = [[[[]] if atom in chs 
                else [] if [1 for hyp in chs if hyp[1] == atom[1]]
                else coind(atom,program,chs.append(query))
               
                for atom in body]
               
               for body in matched_bodies]
    
    return answers


In [ ]:
def collaps(setOfaSetOfPartialAnswers):
    

In [99]:
coind(p,prog1,[])

[[[]]]

## 4. Goal-directed Execution of Answer Set Programs


In [159]:
progTest = [rul1,rul2,[[0,"z"],[]]]
print(progTest)

[[[0, 'p'], [[1, 'q']]], [[0, 'q'], [[1, 'p']]], [[0, 'z'], []]]


In [208]:
def step(program,goals,answer):
    if not goals:
        return ("success", answer)
    elif not [1 for rule in program if rule[0] == goals[0]]:
        return("failR", answer)
    elif [not goals[0][0],goals[0][1]] in answer:
        return ("failA", answer)
    elif goals[0] in answer and not [1 for a in answer if a[0] == 1]:
        return ("failN", answer)
    elif [1 for rule in program if rule[0] == goals[0] and not rule[1]]:
        answer.append(goals[0])
        return (goals[1:],answer)
    elif (goals[0] in answer) and not [1 for a in answer if a[0] == 1]: ### !!! Hier fehlt noch ein OR case
        return (goals[1:],answer)
    elif not [1 for hyp in answer if hyp[1] == goals[0][1]] and [1 for rule in program if rule[0] == goals[0]]:
        return (goals[1:]+ [atom for rule in program if rule[0] == goals[0] for atom in rule[1]][0], answer)
    return 0

In [209]:
step(progTest,[p],[s])

([1, 'q'], [[0, 's']])

In [194]:
step(progTest,[[0,"z"]],[])

([], [[0, 'z']])

In [162]:
step(progTest,[p],[np])

('failA', [[1, 'p']])

In [163]:
step(progTest,[p],[p])

('failN', [[0, 'p']])

In [164]:
step(progTest,[s],[])

('failR', [])

In [165]:
step(progTest,[],[])

('success', [])

In [166]:
print(progTest)

[[[0, 'p'], [[1, 'q']]], [[0, 'q'], [[1, 'p']]], [[0, 'z'], []]]


In [90]:
[if x>2: 9 for x in [1..5]]

SyntaxError: invalid syntax (Temp/ipykernel_10560/2816070095.py, line 1)

## Graph with Or-/And-Nodes

In [ ]:


class OrNode:
    def __init__(self, sign, name, ruleNodes):
        self.name = name
        self.sign = sign
        self.matched_rules = ruleNodes

class AndNode:
    def __init__(self, headAtomNode, bodyAtomNodes):
        self.head = headAtomNode
        self.body = bodyAtomNodes




def top_down(query,program,chs):

    matched_rules = [rule[1] for rule in program if rule[0] == query]
    
    for body in rule_match:
        
    
    
    # root note
    paths = [[query[0]+atom[0], atom] for atom in rule[1]]
    
    # tree search
    while paths:
        
        # aggregates for next expansion
        tentative_expansion = []
        
        # cycle check and return updates
        for path in paths:
            # matches query
            if path[1][1] == query[1]:
                if (path[0] % 2) == 0:
                    elon = True
                else:
                    olon = True
                if elon and olon:
                    return [elon, olon]
            else:
                tentative_expansion.append(path) 
        
        if not tentative_expansion:
            break
            
        # breadth first search
        paths = [[path[0]+atom[0], atom] 
                 for path in tentative_expansion
                 for rule in program if rule[0] == path[1]
                 for atom in rule[1]]
        
    # non-cyclic case
    if olon: 
        return [False, olon]
    else: 
        return [True, olon]